In [1]:
import numpy as np
import tensorflow as tf
import os

## Gather and Inspect Data

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/notebooks/data/MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /notebooks/data/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /notebooks/data/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /notebooks/data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /notebooks/data/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
print(mnist.validation.labels[:1])
print('Training set', mnist.train.images.shape, mnist.train.labels.shape)
print('Validation set', mnist.validation.images.shape, mnist.validation.labels.shape)
print('Test set', mnist.test.images.shape, mnist.test.labels.shape)

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
Training set (55000, 784) (55000, 10)
Validation set (5000, 784) (5000, 10)
Test set (10000, 784) (10000, 10)


## Graphical Reset/Define Layers of Neural Net

In [4]:
tf.reset_default_graph() #First step is to reset graph
keep_prob = tf.placeholder(tf.float32) 

def hidden(data, in_dims, out_dims, name):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([in_dims, out_dims], stddev=0.01), name="W")
        b = tf.Variable(tf.zeros([out_dims]), name="B")
        out = tf.matmul(data, w) + b
        act = tf.nn.relu(out)
        return act        

In [5]:
def out_layer(data, in_dims, out_dims, name):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([in_dims, out_dims], stddev=0.01), name="W")
        b = tf.Variable(tf.zeros(out_dims))
        out = tf.matmul(data, w) + b
        return out

## Define Hyperparameters

In [6]:
batch_size = 100
num_of_epochs = 8
image_size = 28
num_labels = 10
log_dir = '/notebooks/data/demo/'
lr = 1e-2

## Name the Run

In [7]:
run_name = 'PlzWork'

## Input Defnition

In [8]:
x = tf.placeholder(tf.float32, shape=[None, image_size * image_size], name="X")
y_label = tf.placeholder(tf.float32, shape=[None, num_labels], name="Y_label")

image_shaped_input = tf.reshape(x, [-1, image_size, image_size, 1])
tf.summary.image('input', image_shaped_input, 10)

<tf.Tensor 'input:0' shape=() dtype=string>

## Network Definition

In [9]:
hidden1 = hidden(x, image_size * image_size, 100, "hidden1")
hidden2 = hidden(hidden1, 100, 150, "hidden2")
hidden3 = hidden(hidden2, 150, 50, "hidden3")
hidden4 = hidden(hidden3, 50, 25, "hidden4")
drop = tf.nn.dropout(hidden4, keep_prob) # ADDING DROPOUT FOR ALL HIDDEN LAYERS
pred = out_layer(drop, 25, num_labels, "pred")

## Operation Definition

In [10]:
with tf.name_scope("xent"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y_label))
with tf.name_scope("accuracy"):
    correct = tf.equal(tf.argmax(pred, 1), tf.argmax(y_label, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
tf.summary.scalar('acc', accuracy)
tf.summary.scalar('loss', cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



<tf.Tensor 'loss:0' shape=() dtype=string>

In [11]:
with tf.name_scope("train"):
    train_step = tf.train.AdamOptimizer(lr).minimize(cost)

## Initialize Tensorflow Session

In [12]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

## Setup Tensorboard Parameters

In [13]:
summ = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(log_dir + 'train/' + run_name)
train_writer.add_graph(sess.graph)

## Train Model

In [14]:
total_train_indexes = len(mnist.train.images)
for epoch in range(num_of_epochs):
    i = 0
    epoch_loss = 0
    while i < total_train_indexes:
        start = i
        end = i + batch_size

        if end > total_train_indexes:
            end = total_train_indexes

        batch_dataset = mnist.train.images[start:end]
        batch_labels = mnist.train.labels[start:end]

        input_dict = {x: batch_dataset, y_label: batch_labels,  keep_prob: 0.8} # Define dropout rate for training data 0.8
        _, epoch_accuracy, loss, s = sess.run([train_step, accuracy, cost, summ], feed_dict=input_dict)
        epoch_loss += loss
        train_writer.add_summary(s, i + total_train_indexes * epoch)
        i += batch_size
    valid_accuracy, valid_loss = sess.run([accuracy, cost], 
                                                   feed_dict={x: mnist.validation.images, 
                                                              y_label: mnist.validation.labels, keep_prob: 1.0 })  #Validation dropout rate
    
    print('Epoch', epoch + 1, 'completed out of',num_of_epochs,'train_loss:',epoch_loss, 'train_accuracy:', epoch_accuracy)
    print('Epoch', epoch + 1, 'completed out of',num_of_epochs,'valid_loss:',valid_loss, 'valid_accuracy:', valid_accuracy)
    
test_accuracy, test_loss = sess.run([accuracy, cost], feed_dict={x: mnist.test.images, y_label: mnist.test.labels, keep_prob: 1.0}) #Test dropout rate

print("Test accuracy:", test_accuracy, "Test loss:", test_loss)

Epoch 1 completed out of 8 train_loss: 444.6522794365883 train_accuracy: 0.96
Epoch 1 completed out of 8 valid_loss: 0.3211672 valid_accuracy: 0.9268
Epoch 2 completed out of 8 train_loss: 157.63891879376024 train_accuracy: 0.93
Epoch 2 completed out of 8 valid_loss: 0.20030864 valid_accuracy: 0.953
Epoch 3 completed out of 8 train_loss: 122.67538427375257 train_accuracy: 0.96
Epoch 3 completed out of 8 valid_loss: 0.18923081 valid_accuracy: 0.9568
Epoch 4 completed out of 8 train_loss: 108.33007642999291 train_accuracy: 0.96
Epoch 4 completed out of 8 valid_loss: 0.26075068 valid_accuracy: 0.9458
Epoch 5 completed out of 8 train_loss: 100.36086403951049 train_accuracy: 0.99
Epoch 5 completed out of 8 valid_loss: 0.1768498 valid_accuracy: 0.9628
Epoch 6 completed out of 8 train_loss: 89.91976405819878 train_accuracy: 0.97
Epoch 6 completed out of 8 valid_loss: 0.18138185 valid_accuracy: 0.9618
Epoch 7 completed out of 8 train_loss: 88.0655701829819 train_accuracy: 0.99
Epoch 7 complete